In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/10/09 16:19
# @Author  : Wang Yujia
# @File    : SA_for_PT_model.ipynb
# @Description : Parameter estimation for PT_model using Simulated Annealing

# 0. What for

# 1. Preparations
1. infer参数一是需要data，二是需要把p表示出来才能写出来loss func
2. data来自`data_selected_path`

## 1.1 全局设置
1. 除了表示uniq auction的features，还引入了
    - 'cnt_uniq':表示paper里的Loss function公式里的A

In [11]:
# data path
data_selected_path = "../data/info_asymm/datawithnp_asc_symmetry_2_selected.csv"
# optimized parameters' saving path:
params_opitim_path = "../data/SA_PT/params_opitim.csv"

# for PT
alpha = 1
delta = 1
labda = 2.25
# features that GT need
features_GT = ['product_id','bidincrement','bidfee','retail']
features_GT_infer = ['cnt_uniq']

# for SA
# initial params
table_5_M = [0.025,0.85,3.72]
# lower/ upper bound
lb = [-0.3,0.01,0.01]
ub = [0.3, 2, 16]

import numpy as np
import pandas as pd
import sympy
import seaborn as sns
from tqdm.notebook import tqdm
from sko.SA import SABoltzmann
import matplotlib.pyplot as plt
import sys

## 1.2 data 读取
1. 读取data以做SA
2. 提取出来`data_key`，以及其他计算需要的features

In [17]:
data = pd.read_csv(data_selected_path, encoding="utf-8")
data_key = data[features_GT].copy()
data_key.drop_duplicates(inplace=True)

B = np.array(data.bidfee)          # bid fee (cent to dollar)
D = np.array(data.bidincrement)    # bid increment (cent to dollar)
V = np.array(data.retail)               # valuation
# 需要计算`N_uniq_auction`组setting下的结果
N_uniq_auction= data_key.shape[0]

print("For PT model, there are *{}* settings waiting to be inferred.".format(N_uniq_auction))

For PT model, there are *1303* settings waiting to be inferred.


## 1.3 functions about 'key'

In [3]:
# get key from i in 'data_key'
def get_key_from_index(i):
   key_i = data_key.iloc[i,:]
   return key_i

#features_GT = ['product_id','bidincrement','bidfee','retail']
def select_data_fromkey(key_i_str):
    return data[(data['product_id'] == key_i_str[0]) & (data['bidincrement'] == key_i_str[1]) & (data['bidfee'] == key_i_str[2]) & (data['retail'] == key_i_str[3])].copy()

# 2. PT model
## 2.1 prob. weighting func
1. 根据Eq(5)

In [61]:
def OMEGA(p,delta):
    return p**delta * ((p**delta + (1-p)**delta)**(-1/delta))

## 2.2 C_{t-1}
1. 根据5.1.2

In [4]:
def C(t,b):
    return 0.2*t*b

## 2.3 value functions
1. 根据Eq(7)-(9)
2. 注意这里把(-labda)(1-sympy.E**(alpha*x))/alpha的`labda`拿到外面去了，方便写

In [5]:
# valuation function
def f(x, alpha):
    return (1-sympy.E**(-alpha*x))/alpha
    # when x < 0, in fact, it shoule be : (-labda)*(1-sympy.E**(alpha*x))/alpha

## 2.4 Equi. condition
1. 根据Eq(6)
2. 注意分辨怎么代入上面的公式

In [80]:
def f_Equi(t,v,d,b,alpha,labda,delta):
    u = sympy.Symbol('u')

    tmp = v-d*t-C(t-1,b) - b

    func_1 = (labda * f(x=C(t-1, b), alpha=alpha) - labda * OMEGA(u, delta) * f(x=(C(t-1, b) + b), alpha=alpha) + OMEGA(1-u, delta) * f(tmp, alpha))
    func_2 = (-f(x=C(t-1, b), alpha=alpha) + OMEGA(u, delta) * f(x=(C(t-1, b) + b), alpha=alpha) + (1 - OMEGA(u, delta)) * f(-tmp, alpha))

    if(tmp >= 0):
        return sympy.nsolve(func_1,(0,1),solver='bisect', verify=False)
    else:
        return sympy.nsolve(func_2,(0,1),solver='bisect', verify=False)

# 3. SA
## 3.1 define loss function
1. loss function: NLL for auctions with same `features_GT`
2.

In [22]:
def loss_func(params):

    alpha = params[0]
    delta = params[1]
    labda = params[2]

    # solve for U from Equi. condt.
    U_i = [0] * (max_T + 1)
    U_i[0] = 1

    for t in tqdm(range(1,max_T+1),desc="solve for U"):

        U_i[t] = f_Equi(t, v, d, b, alpha, labda, delta)

    # calculate NLL under this auction setting & PT params
    nll = 0
    if(U_i[0]==1):
        U_i.pop(0)            # because U_i[0]=1
    U_tmp_df = pd.DataFrame(U_i, index=np.arange(0, U_i.__len__()), columns=['U'], dtype=float)

    for idx in range(0,data_i.shape[0]):
        # sum up the log prob among all durations of this auction
        nll += ( np.sum(U_tmp_df[0:(T_i-1)][:].apply(np.log,axis=1)) + np.log(1-U_tmp_df.iat[(T_i-1),0]) )* cnt_n_2_i[idx]

    return float(-nll)

## 3.2 do SA
1. 要对每一个setting做一次infer == 对每一个setting执行一次SA。
    - 可以并行吗？YES
2. 具体的：对每个setting `i`
    - 每一个setting `i` 可以提取出来一个`data_i`，代表所有auction
    - 每一个`data_i`中的`cnt_uniq`，也就是`A`，是相同的，表示setting `i` 进行的拍卖总次数.【但是这个`A`在计算loss的时候派不上用场】
    - `N`表示duration，因此paper公式里的$T_a$即`N[a]`
    - 因此有`A = sum(data_i['cnt_n_2'])`，其中的'cnt_n_2'表示了该行对应的`duration=N`发生的次数
    - 按照上文，求解`U[i]_t` which is a array with shape of (max(N)),也就是求解paper里的`p_t`
    - 求nll时，记得

In [25]:
params_opitim = pd.DataFrame(columns=['key_idx','params'])
# best_x = [1,2,3]
# i = 0
# df_tmp = pd.DataFrame([[i,best_x]],columns=['key_idx','params'])
# params_opitim = params_opitim.append(df_tmp,ignore_index=True)  # ignore_index=True could help in rearranging index

In [ ]:
# Perform SA respectively for all settings
for i in range(0,N_uniq_auction):

    # get i_th data_key
    key_i = get_key_from_index(i)
    # extract data with same `key_i` into a table
    data_i = select_data_fromkey(key_i)
    data_i.reset_index(drop=True,inplace=True)

    T_i = data_i['N'].astype(int)          # auction duration sequence
    max_T = int(max(T_i))                  # max duration value

    cnt_n_2_i = data_i['cnt_n_2'].astype(int)       # Number of occurrences of different durations
    # for a certain auction(like 'data_i'), 'cnt_uniq' should be all the same
    A_i = int(data_i['cnt_uniq'].unique())
    assert(A_i == sum(cnt_n_2_i),"'cnt_uniq' does not match with sum of 'cnt_n_2'!")

    v = float(data_i['retail'].unique())
    d = float(data_i['bidincrement'].unique())
    b = float(data_i['bidfee'].unique())

    # calculate NLL
    print("> Initilizing SA....... \n")
    sa_boltzmann = SABoltzmann(func=loss_func, x0=table_5_M, T_max=1000, T_min=1e-5, learn_rate=0.01, L=50, max_stay_counter=50,
                            lb=lb, ub=ub)

    print("> Now do SA....... \n")
    best_x, best_y = sa_boltzmann.run()

    print("> SA ENDS....... \n")

    # draw the pic of NLL Loss in SA
    plt.title("In {0} iteration of {1}, the T(NLL) in SA".format(i,N_uniq_auction))
    plt.xlabel("iter_cycle")
    plt.ylabel("T of SA")
    sns.lineplot(x = np.arange(0,sa_boltzmann.iter_cycle+1),y=np.array(sa_boltzmann.generation_best_Y))

    # append the opitimized params into the df
    df_tmp = pd.DataFrame([[i,best_x]],columns=['key_idx','params'])
    params_opitim = params_opitim.append(df_tmp,ignore_index=True)  # ignore_index=True could help in rearranging index

In [ ]:
# save 'params_opitim' for later check
params_opitim.to_csv(params_opitim_path, header=True, encoding="utf-8",index=False)

## 3.3
1. 得到`params_opitim`之后，可以对不同的auction settings做generate了
2. generate过程无非是求u-->p，u的代码在上面loss func里写过了